# *fiasco* Tutorial
This notebook shows how to parse the CHIANTI atomic database, build the database into an HDF5 file, and then access the data by streaming it from the HDF5 file.

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.utils.console import ProgressBar
import fiasco
import fiasco.io

%matplotlib inline

## Setup
1. [Download and install the `fiasco` package and the associated dependencies](https://github.com/wtbarnes/fiasco). 
2. [Download](http://www.chiantidatabase.org/chianti.html) the CHIANTI atomic database (if not already installed through SSW)
3. Point the fiasco package to the database using
  1. the usual XUVTOP environment variable
  2. setting the path in the `$HOME/.fiasco/fiascorc` file, e.g.
  ```yaml
  [database]
  dbase_root=/my/path/to/chianti/dbase
  ```
  
However you've configured it, the root of the CHIANTI database should be reachable as,

In [17]:
fiasco.defaults['chianti_dbase_root']

'/Users/willbarnes/Documents/Rice/Research/ssw/packages/chianti/dbase'

## Parsing the Raw CHIANTI Database
This package provides a convenient parser interface for any CHIANTI filetype. The parser uses a factory pattern to create a different subclass based on the file extension. This allows the interface to multiple different files to be the same and provides the needed metadata (title/description and units) to each column in each file.

First, implement a parser for a specific CHIANTI file, say `h_1.elvlc` which contains the energy levels for H I.

In [2]:
parser = fiasco.io.Parser('h_1.elvlc')

In [3]:
parser

Notice that this returns a parser specifically for the `.elvlc` filetype. Now, call `.parse()` on the parser to return the data in this file as an [Astropy quantity table](http://docs.astropy.org/en/stable/table/mixin_columns.html#quantity-and-qtable) with associated metadata.

In [5]:
table = parser.parse()

In [6]:
table

level index,configuration,level label,multiplicity,orbital angular momentum,total angular momentum,observed energy,theoretical energy
,,,,,,1 / cm,1 / cm
int64,str2,str1,int64,str1,float64,float64,float64
1,1s,,2,S,0.5,0.0,0.0
2,2s,,2,S,0.5,82258.956,82303.0
3,2p,,2,P,0.5,82258.921,82303.0
4,2p,,2,P,1.5,82259.287,82303.0
5,3s,,2,S,0.5,97492.224,97544.0
6,3p,,2,P,0.5,97492.213,97544.0
7,3p,,2,P,1.5,97492.321,97544.0
8,3d,,2,D,1.5,97492.321,97544.0


In [8]:
table['observed energy']

<Quantity [      0.   ,  82258.956,  82258.921,  82259.287,  97492.224,
             97492.213,  97492.321,  97492.321,  97492.357, 102823.855,
            102823.851, 102823.896, 102823.896, 102823.911, 102823.911,
            102823.919, 105291.633, 105291.631, 105291.654, 105291.654,
            105291.662, 105291.662, 105291.666, 105291.666, 105291.668] 1 / cm>

The `.meta` dictionary holds additional information, including the original file footer.

In [9]:
table.meta

OrderedDict([('footer',
              "%filename: h_1.elvlc\n%observed energy levels: Fuhr et al, 1999, NIST Atomic Spectra Database Version 2.0\n%produced as part of the Arcetri/Cambridge/NRL 'CHIANTI' atomic data base collaboration\n%\n%  Ken Dere  May 3 2001\n"),
             ('chianti_version', '8.0.2'),
             ('element', 'h'),
             ('ion', 'h_1')])

In [11]:
print(table.meta['footer'])

%filename: h_1.elvlc
%observed energy levels: Fuhr et al, 1999, NIST Atomic Spectra Database Version 2.0
%produced as part of the Arcetri/Cambridge/NRL 'CHIANTI' atomic data base collaboration
%
%  Ken Dere  May 3 2001



Notice that this same interface persists for every filetype and for every ion...

In [12]:
fiasco.io.Parser('h_1.wgfa').parse()

lower level index,upper level index,transition wavelength,oscillator strength,radiative decay rate,lower level label,upper level label
,,Angstrom,,1 / s,,
int64,int64,float64,float64,float64,str8,str8
1,2,1215.675,0.0,2.496e-06,1s 2S1/2,2s 2S1/2
1,2,0.0,0.0,8.229,1s 2S1/2,2s 2S1/2
1,19,949.745,0.0186,34400000.0,1s 2S1/2,5p 2P3/2
1,18,949.745,0.0093,34400000.0,1s 2S1/2,5p 2P1/2
1,12,972.538,0.0387,68200000.0,1s 2S1/2,4p 2P3/2
1,11,972.539,0.0193,68200000.0,1s 2S1/2,4p 2P1/2
1,7,1025.724,0.105,167000000.0,1s 2S1/2,3p 2P3/2
1,6,1025.725,0.0527,167000000.0,1s 2S1/2,3p 2P1/2


In [13]:
fiasco.io.Parser('fe_11.scups').parse()

lower level index,upper level index,delta energy,oscillator strength,high-temperature limit,number of scaled temperatures,Burgess-Tully scaling type,Burgess-Tully scaling parameter,Burgess-Tully scaled temperatures [20],Burgess-Tully scaled effective collision strengths [20]
,,Ry,,1 / Ry,,,,,
int64,int64,float64,float64,float64,int64,int64,float64,float64,float64
1,2,0.1154,-1.0,1e-06,20,2,37.43,0.0 .. 1.0,2.614 .. 1e-06
1,3,0.1304,-1.0,1e-06,20,2,33.13,0.0 .. 1.0,0.7307 .. 1e-06
1,4,0.3439,-1.0,1e-06,20,2,12.56,0.0 .. 1.0,1.345 .. 1e-06
1,5,0.7366,-1.0,1e-06,20,2,5.864,0.0 .. 1.0,0.5576 .. 1e-06
1,6,2.584,0.2164,0.335,20,1,1.495,0.0 .. 1.0,1.826 .. 0.335
1,7,2.671,0.07814,0.117,20,1,1.477,0.0 .. 1.0,0.7161 .. 0.117
1,8,2.726,-1.0,1e-06,20,2,1.585,0.0 .. 1.0,0.1226 .. 1e-06
1,9,3.297,0.009563,0.0116,20,1,1.373,0.0 .. 1.0,0.6907 .. 0.0116


The same interface can be used to read the abundance, ionization equilibrium, and ionization potential files too.

In [15]:
fiasco.io.Parser('sun_coronal_1992_feldman.abund').parse()

atomic number,abundance relative to H,element
,,
int64,float64,str2
1,1.0,H
2,0.07943282347242822,He
6,0.0003890451449942805,C
7,0.0001,N
8,0.0007762471166286928,O
10,0.00012022644346174131,Ne
11,8.51138038202376e-06,Na
12,0.00014125375446227554,Mg


In [14]:
fiasco.io.Parser('chianti.ioneq').parse()

atomic number,ion,temperature [101],ionization fraction [101]
,,K,
int64,int64,float64,float64
1,1,10000.0 .. 1000000000.0,0.9983 .. 1.664e-10
1,2,10000.0 .. 1000000000.0,0.001693 .. 1.0
2,1,10000.0 .. 1000000000.0,1.0 .. 5.607e-17
2,2,10000.0 .. 1000000000.0,1.904e-09 .. 1.093e-08
2,3,10000.0 .. 1000000000.0,1.592e-34 .. 1.0
3,1,10000.0 .. 1000000000.0,0.004061 .. 1.014e-23
3,2,10000.0 .. 1000000000.0,0.9959 .. 1.52e-14
3,3,10000.0 .. 1000000000.0,0.0 .. 1.46e-07


In [16]:
fiasco.io.Parser('chianti.ip').parse()

atomic number,ion,ionization potential
,,1 / cm
int64,int64,float64
1,1,109678.7737
2,1,198310.77227
2,2,438908.8863
3,1,43487.15
3,2,610079.0
3,3,987661.027
4,1,75192.64
4,2,146882.86


fiasco uses a subclass registry in the parser factory to determine the right subclass for the provided extension. This means you can write your own parser for a specific filetype by just subclassing the `fiasco.io.GenericParser` base class and adding a `filetype` attribute.

In [5]:
class JpegParser(fiasco.io.GenericParser):
    filetype='jpeg'
    def __init__(self,*args,**kwargs):
        print('A custom registered parser')
    def parse(self):
        print('Nothing to parse here')

In [6]:
fiasco.io.Parser('hello.jpeg').parse()

A custom registered parser
Nothing to parse here


## Building the HDF5 File
To avoid having to repeatedly read from the plain text files, we want to organize the CHIANTI data into a more convenient format, e.g. HDF5. Using something like HDF5 also means we can stream only the parts of the data that we need out of the file and avoid unnecessary overhead. The `Parser` object has a method `to_hdf5()` so building the HDF databse is just a matter of calling this method on each file in the CHIANTI database.

Eventually, this will be a function built into the package. For now, we'll just do it externally. If you want to store the HDF5 file somewhere particular, you should set this in the `fiascorc` file, e.g.

```yaml
[database]
chianti_hdf5_dbase_root=/custom/path/to/chianti.hdf5
```

If no location is set, the default is `$HOME/.fiasco/chianti_dbase.h5`.

In [20]:
# collect all the filenames
# this is cumbersome and there is a better way.
skip_dirs = ['abundance','ip','ancillary_data','continuum','dem','ioneq','masterlist']
all_files = []
for topdir,subdir,files in os.walk(fiasco.defaults['chianti_dbase_root']):
    if np.any([sd in topdir for sd in skip_dirs]):
        continue
    if topdir == fiasco.defaults['chianti_dbase_root']:
        continue
    files = [file for file in files if file[0] != '.']
    all_files += files
abund_files = [file for file in os.listdir(os.path.join(fiasco.defaults['chianti_dbase_root'],'abundance')) 
               if file[0]!='.' and os.path.isfile(os.path.join(fiasco.defaults['chianti_dbase_root'],'abundance',file))]
abund_files += [os.path.join('version_3',file) for file in os.listdir(os.path.join(fiasco.defaults['chianti_dbase_root'],'abundance','version_3'))
                if file[0]!='.']
ioneq_files = [file for file in os.listdir(os.path.join(fiasco.defaults['chianti_dbase_root'],'ioneq')) 
               if file[0]!='.' and os.path.isfile(os.path.join(fiasco.defaults['chianti_dbase_root'],'ioneq',file))]
ioneq_files += [os.path.join('deprecated',file) for file in os.listdir(os.path.join(fiasco.defaults['chianti_dbase_root'],'ioneq','deprecated'))
                if file[0]!='.']
ip_files = ['chianti.ip']
all_files += abund_files + ioneq_files + ip_files
all_files = [af for af in all_files if '_all.' not in af and len(af.split('.'))==2]

In [ ]:
with ProgressBar(len(all_files),ipython_widget=True,) as progress:
    with h5py.File(fiasco.defaults['chianti_hdf5_dbase_root'],'a') as hf:
        for file in all_files:
            parser = fiasco.io.Parser(file)
            df = parser.parse()
            parser.to_hdf5(hf,df)
            progress.update()

## Accessing CHIANTI Data from HDF5
Now that we've parsed the data and built it into an HDF5 file, we need a way to access it. `fiasco` implements a class `IonBase` for accessing all of the data associated with a particular ion.

The idea behind this object is to provide a general base layer for accessing CHIANTI data. More specific objects can then be built on top of this data access layer. Additionally, this provides a nice base layer for users/other applications to build on top if they need to access CHIANTI data but don't want the overhead or clutter of the specific CHIANTI methods, e.g. all the different needed calculations.

In [8]:
h1_ion = fiasco.IonBase('h_1')

This class has an attribute for each filetype.

In [9]:
h1_ion.elvlc

h/h_1/elvlc
        
Fields
------
configuration  -- [description]
level index  -- [description]
level label  -- [description]
multiplicity  -- [description]
observed energy (1 / cm) -- [description]
orbital angular momentum  -- [description]
theoretical energy (1 / cm) -- [description]
total angular momentum  -- [description]

Footer
------
%filename: h_1.elvlc
%observed energy levels: Fuhr et al, 1999, NIST Atomic Spectra Database Version 2.0
%produced as part of the Arcetri/Cambridge/NRL 'CHIANTI' atomic data base collaboration
%
%  Ken Dere  May 3 2001

        

In [10]:
h1_ion.wgfa

h/h_1/wgfa
        
Fields
------
lower level index  -- [description]
lower level label  -- [description]
oscillator strength  -- [description]
radiative decay rate (1 / s) -- [description]
transition wavelength (Angstrom) -- [description]
upper level index  -- [description]
upper level label  -- [description]

Footer
------
%filename: h_1.wgfa
% observed energy levels: Fuhr et al, 1999, NIST Atomic Spectra Database Version 2.0,
%oscillator strengths: Wiese,W.L., Smith,M.W., Glennon,B.M., 1966, Atomic Transition Probabilities, NSRDS-NBS-4
% A values (1s-2s): Parpia,F.A., Johnson,W.R., 1982, Phys.Rev.A, 26, 1142.
%comment: Wavelengths have been calculated the original energy levels from NIST, that include more
          decimal figures than allowed by CHIANTI format.
%produced as part of the Arcetri/Cambridge/NRL 'CHIANTI' atomic data base collaboration
%
%  Ken Dere  May 2001
 -1

        

Notice that the `__repr__` is overridden to display the necessary metadata about each file for a given ion. Additionally, the `__getitem__` has been overriden to return the "columns" of that attribute from the HDF5 file. This makes accessing the data much easier and then also provides the original sources for the data as listed in the footer of the original data file.

In [11]:
h1_ion.elvlc['observed energy']

<Quantity [      0.   ,  82258.956,  82258.921,  82259.287,  97492.224,
             97492.213,  97492.321,  97492.321,  97492.357, 102823.855,
            102823.851, 102823.896, 102823.896, 102823.911, 102823.911,
            102823.919, 105291.633, 105291.631, 105291.654, 105291.654,
            105291.662, 105291.662, 105291.666, 105291.666, 105291.668] 1 / cm>

In [12]:
h1_ion.wgfa['transition wavelength']

<Quantity [  1215.675,     0.   ,   949.745,   949.745,   972.538,
              972.539,  1025.724,  1025.725,  1215.67 ,  1215.676,
             4341.655,  4341.659,  4341.661,  4341.665,  4341.723,
             4341.724,  4341.729,  4862.645,  4862.652,  4862.656,
             4862.664,  4862.729,  4862.732,  4862.743,  6564.536,
             6564.549,  6564.576,  6564.596,  6564.677,  6564.694,
             6564.734, 12821.448, 12821.474, 12821.486, 12821.513,
            12821.615, 12821.615, 12821.628, 12821.628, 12821.667,
            12821.667, 12821.68 , 12821.68 , 12821.692, 18755.822,
            18755.877, 18755.988, 18756.043, 18756.152, 18756.152,
            18756.207, 18756.207, 18756.262, 18756.289, 18756.344,
            18756.373, 18756.373, 40521.922, 40521.922, 40522.305,
            40522.305, 40522.562, 40522.562, 40522.691, 40522.691,
            40522.816, 40522.816, 40522.816, 40522.816, 40522.816,
            40522.945, 40522.945, 40522.945, 40522.945, 40523.

If a particular filetype does not exist for that ion, `None` is returned.

**Note: Each `__getitem__` is a read on the HDF5 file. While this may result in some additional overhead, it means that we don't pay in having to store information we don't need. This is especially advantagious for particularly large files. For internal routines, **

For the abundance, ionization potential, and ionization equilibrium data, the keys are the different available datasets.

In [16]:
h1_ion.abundance

h/abundance
        
Fields
------
allen  -- [description]
allen_minorions  -- [description]
cosmic_1973_allen  -- [description]
feldman  -- [description]
grevesse_anders  -- [description]
grevesse_sauval98  -- [description]
meyer_coronal  -- [description]
photospheric_may97  -- [description]
sun_coronal_1992_feldman  -- [description]
sun_coronal_1992_feldman_ext  -- [description]
sun_coronal_1999_fludra  -- [description]
sun_coronal_1999_fludra_ext  -- [description]
sun_coronal_2012_schmelz  -- [description]
sun_coronal_2012_schmelz_ext  -- [description]
sun_photospheric_1998_grevesse  -- [description]
sun_photospheric_2007_grevesse  -- [description]
sun_photospheric_2009_asplund  -- [description]
sun_photospheric_2009_lodders  -- [description]
sun_photospheric_2011_caffau  -- [description]
unity  -- [description]
waljeski  -- [description]

Footer
------
cosmic_1973_allen
------------------
%filename: cosmic_allen_1973.abund
%elemental abundances: Allen, C.W., 1973, Astrophysical Qua

In [14]:
h1_ion.ip

h/h_1/ip
        
Fields
------
chianti (1 / cm) -- [description]

Footer
------
chianti
------------------
%filename: chianti.ip
%ionization potentials: Martin, W.C., Sugar, J., Musgrove, A., Dalton, G.R.,
    1995, NIST Database for Atomic Spectroscopy, Version 1.0, NIST Standard Reference Database 61
    and Version 2.0 http://physics.nist.gov/cgi-bin/AtData/main_asd
%ionization postentials: additional data for Cu and Zn from Biemeont, E., Fremat, Y., Quinet, P., 1999, ADNDT, 71, 117
%ionization potentials: Erickson, G.W., 1977, J. Phys. Chem. Ref. Data, 6, 831
%ionization potentials: C.E. Moore, 1970, NSRDS-NBS 
%ionization potentials: Boiko V.A., Pal'chikov V.G., Skobelev I.Yu., Faenov A.Ya., Boiko V.A., Pal'chikov V.G., 
    Skobelev I.Yu., Faenov A.Ya. from - http://spectr-w3.snz.ru/index.phtml
% produced as part of the Arcetri/Cambridge/NRL 'CHIANTI' atomic data base collaboration
%
%   K.P.Dere 2006
 -1

        
        

In [17]:
h1_ion.ioneq

h/h_1/ioneq
        
Fields
------
arnaud_raymond  -- [description]
arnaud_raymond_ext  -- [description]
arnaud_rothenflug  -- [description]
arnaud_rothenflug_ext  -- [description]
bryans_etal_06  -- [description]
bryans_etal_09  -- [description]
chianti  -- [description]
chianti_version6  -- [description]
mazzotta_etal  -- [description]
mazzotta_etal_9  -- [description]
mazzotta_etal_ext  -- [description]
shull_steenberg  -- [description]
shull_steenberg_ext  -- [description]

Footer
------

        

In [18]:
h1_ion.ioneq['shull_steenberg']

h/h_1/ioneq/shull_steenberg
        
Fields
------
ionization fraction  -- [description]
temperature (K) -- [description]

Footer
------
%filename: shull_steenberg.ioneq
%ionization equilibrium: Shull & Steenberg 1982, ApJSS, 48, 95
%ionization equilibrium H and He: Arnaud, M., Rothenflug, R., 1986, AASS, 60, 425
% produced as part of the Arcetri/Cambridge/NRL 'CHIANTI' atomic data base collaboration
%
% Enrico Landi Jan 2003
 -1

        